In [4]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib
import statannotations
from statannotations.Annotator import Annotator
import math
import os
#pip install --upgrade seaborn==0.11.2
#REQUIERD VERSION 0.11.2

# Functions

In [5]:
def FAindex6(rightMeasurements, leftMeasurements):
    # Based on: https://www.annualreviews.org/doi/pdf/10.1146/annurev.es.17.110186.002135
    # Page 395, table 1
    areaNormalized = rightMeasurements;
    
    for numMeas in range(len(rightMeasurements)):
        a_i = rightMeasurements[numMeas] - leftMeasurements[numMeas];
        avg = (rightMeasurements[numMeas] + leftMeasurements[numMeas])/2;
        areaNormalized[numMeas] = a_i / avg;
        
    res = np.var(areaNormalized)
    return [res, areaNormalized]
    

# Defining initial conditions

In [6]:
# Defining initial folders
whoIsRunning = 'Pablo' # or 'Katherine'

if whoIsRunning == 'Pablo':
    inputFolder = 'data'
    #conditionsFolder = ['YW/1g', 'sqhAX3_sqhGFP_ubiRFPCAAX/1g', 'sqhAX3_sqhGFP_ubiRFPCAAX/33RPM', 'sqhAX3_sqhGFP_ubiRFPCAAX/5RPM']
    #conditions = ['YW_Still', 'sqhAX3_Still', 'sqhAX3_33RPM', 'sqhAX3_5RPM']
    #conditionsFolder = ['YW/1g', 'sqhAX3_sqhGFP_ubiRFPCAAX/1g']
    #conditions = ['YW_Still', 'sqhAX3_Still']
    #conditionsFolder = ['sqhAX3_sqhGFP_ubiRFPCAAX/1g', 'sqhAX3_sqhGFP_ubiRFPCAAX/33RPM', 'sqhAX3_sqhGFP_ubiRFPCAAX/5RPM']
    #conditions = ['sqhAX3_Still', 'sqhAX3_33RPM', 'sqhAX3_5RPM']
    conditionsFolder = ['sqhAX3_sqhGFP_ubiRFPCAAX/1g', 'sqhAX3_sqhGFP_ubiRFPCAAX/33RPM']
    conditions = ['sqhAX3_Still', 'sqhAX3_33RPM']
    sexes = {'F', 'M', ''}
    typesOfAnalysis = {'Output_area.dat', 'Output_vLength.dat', 'Output_Landmarks.dat'}
elif whoIsRunning == 'Katherine':
    inputFolder = '/Volumes/lmcb4/ym_giulia_s1019_f1222/Katherine_project/Quantitative_wing_analysis/AxioPlan_Images'
    conditionsFolder = ['18Nov2022', '16Dec2022/yw_0.1mm/macro_output', '13Jan2023/yw_0.2mm/macro_output',
                    '26Jan2023/yw_0.3mm/macro_output', '26Jan2023/yw_0.5mm/macro_output']
    conditions = ['straight', '0.1', '0.2', '0.3', '0.5']
    typesOfAnalysis = {'CPR_output/Fluctuating_asymmetry/Output_area.dat', 'CPR_output/Fluctuating_asymmetry/Output_vLength.dat', 'CPR_output/Fluctuating_asymmetry/Output_Landmarks.dat'}
    sexes = {'F', 'M', ''}

features = {'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', \
            'L10', 'L11', 'L12', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', \
           'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', \
            'A4_A5', 'A1_A2_A3', 'A6_A7', \
           'L9_L10_L11', 'L7_L8', 'L12_L13', \
           'wingWidth'}

In [7]:
outputFolder = inputFolder+'/Results' + "-".join(conditions)

if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

## Loading data

In [8]:
data = []
for numCondition in range(len(conditionsFolder)):
    dataCondition = []
    for typeOfAnalysis in typesOfAnalysis:
        filepath = inputFolder+'/'+conditionsFolder[numCondition]+'/'+typeOfAnalysis
        print(filepath)
        new_df = pd.read_csv(filepath,sep='\t')
        new_df['condition'] = conditions[numCondition]
        dataCondition.append(new_df)

    dfCondition = pd.concat(dataCondition, axis=1);
    dfCondition = dfCondition.T.drop_duplicates().T;
    data.append(dfCondition)

df_analysis = pd.concat(data);

data/sqhAX3_sqhGFP_ubiRFPCAAX/1g/Output_Landmarks.dat


FileNotFoundError: [Errno 2] No such file or directory: 'data/sqhAX3_sqhGFP_ubiRFPCAAX/1g/Output_Landmarks.dat'

## Compute measurements

### Compute new measurements

In [ ]:
# Width of wing (see Roshan paper)
distance2Points = (df_analysis['x1'] - df_analysis['x4'])**2 + (df_analysis['y1'] - df_analysis['y4'])**2
wingWidth = distance2Points.apply(lambda x: math.sqrt(x))
df_analysis['wingWidth'] = wingWidth

In [ ]:
results = [];
individualResults = [];
for numCondition in range(len(conditions)):
    condition = conditions[numCondition]
    
    for sex in sexes:
        generalCondition = np.array(df_analysis['condition'] == condition);
        
        if np.array(df_analysis['CPFile'].str.contains('_F_')).any():
            if sex == '':
                generalCondition = generalCondition & np.array(df_analysis['CPFile'].str.contains(''))
            else:
                generalCondition = generalCondition & np.array(df_analysis['CPFile'].str.contains('_'+sex+'_'))
            
            rightWings = df_analysis[np.array(df_analysis['CPFile'].str.contains('_R')) & generalCondition];
            leftWings = df_analysis[np.array(df_analysis['CPFile'].str.contains('_L')) & generalCondition];
        else:
            generalCondition = generalCondition & np.array(df_analysis['Sex'].str.contains(sex))
            
            rightWings = df_analysis[np.array(df_analysis['Tags'].str.contains('right')) & generalCondition]
            leftWings = df_analysis[np.array(df_analysis['Tags'].str.contains('left')) & generalCondition]

        # Quantify
        [FAindex6_Area, areaNormalized] = FAindex6(np.array(rightWings['AWing']), np.array(leftWings['AWing']))
        [FAindex6_Width, widthNormalized] = FAindex6(np.array(rightWings['wingWidth']), np.array(leftWings['wingWidth']))
        
        # Measures normalised by area of the wing
        if sex != '':
            conditionsRepeated_left = np.matlib.repmat([condition, sex, 'left'], leftWings.shape[0], 1)
            conditionsRepeated_right = np.matlib.repmat([condition, sex, 'right'], rightWings.shape[0], 1)
        else:
            conditionsRepeated_left = np.matlib.repmat([condition, 'All', 'left'], leftWings.shape[0], 1)
            conditionsRepeated_right = np.matlib.repmat([condition, 'All', 'right'], rightWings.shape[0], 1)
            
        for feature in features:
            if feature in leftWings.columns:
                wingValues_left = np.array(100*leftWings[feature]/leftWings['AWing']);
                wingValues_right = np.array(100*rightWings[feature]/rightWings['AWing']);
            else:
                addFeatures = feature.split('_')
                addingFeatures = leftWings[addFeatures]
                wingValues_left = np.array(100*addingFeatures.sum(axis=1)/leftWings['AWing']);
                addingFeatures = rightWings[addFeatures]
                wingValues_right = np.array(100*addingFeatures.sum(axis=1)/rightWings['AWing']);
                
            conditionsRepeated_left = np.column_stack((conditionsRepeated_left, wingValues_left))
            conditionsRepeated_right = np.column_stack((conditionsRepeated_right, wingValues_right))

        conditionsRepeated_left = np.column_stack((conditionsRepeated_left, leftWings['AWing']))
        conditionsRepeated_right = np.column_stack((conditionsRepeated_right, rightWings['AWing']))
            
        columnNames = np.append(['condition', 'Sex', 'side'], numpy.array(list(features)))
        columnNames = np.append(columnNames, ['AWing'])
        
        newIndividualResults_left = pd.DataFrame(conditionsRepeated_left, columns=columnNames)
        newIndividualResults_right = pd.DataFrame(conditionsRepeated_right, columns=columnNames)
        individualResults.append(newIndividualResults_left)
        individualResults.append(newIndividualResults_right)
        
        # Save results into a DF
        newResult_df = pd.DataFrame([[condition, sex, sum(generalCondition)/2, FAindex6_Area, areaNormalized, FAindex6_Width]], \
                       columns=['condition', 'Sex', 'n', 'FAindex_Area', 'areaNormalised_LR', 'FAindex_Width'])
        results.append(newResult_df)

individualResults_df = pd.concat(individualResults)
results_df = pd.concat(results)

features_array = np.array(list(features))
features_array = np.append(features_array, 'AWing')
individualResults_df[features_array] = individualResults_df[features_array].apply(pd.to_numeric)

In [ ]:
# Sort to show them in the correct order
#individualResults_df = individualResults_df.sort_values(by=['Sex'])

## Wing features from Wings4 normalised by its total wing area

In [ ]:
if whoIsRunning == 'Pablo':
    pairs=[#(("YW_Still", "F"), ("YW_Still", "M")),
           #(("sqhAX3_Still", "F"), ("sqhAX3_Still", "M")),
           #(("sqhAX3_5RPM", "F"), ("sqhAX3_5RPM", "M")),
           #(("sqhAX3_33RPM", "F"), ("sqhAX3_33RPM", "M")),
           #(("YW_Still", "F"), ("sqhAX3_Still", "F")),
           #(("YW_Still", "M"), ("sqhAX3_Still", "M")),
           #(("YW_Still", "All"), ("sqhAX3_Still", "All"))
           #(("sqhAX3_5RPM", "All"), ("sqhAX3_Still", "All")),
           #(("sqhAX3_5RPM", "F"), ("sqhAX3_Still", "F")),
           #(("sqhAX3_5RPM", "M"), ("sqhAX3_Still", "M")),
           #(("sqhAX3_33RPM", "All"), ("sqhAX3_Still", "All")),
           #(("sqhAX3_33RPM", "F"), ("sqhAX3_Still", "F")),
           #(("sqhAX3_33RPM", "M"), ("sqhAX3_Still", "M")),
           #(("sqhAX3_33RPM", "All"), ("sqhAX3_5RPM", "All")),
           #(("sqhAX3_33RPM", "F"), ("sqhAX3_5RPM", "F")),
           #(("sqhAX3_33RPM", "M"), ("sqhAX3_5RPM", "M")),
          ]
else:
    #['straight', '0.1', '0.2', '0.3', '0.5']
    pairs=[(("straight", "F"), ("straight", "M")),
           (("straight", "F"), ("0.1", "F")),
           (("straight", "M"), ("0.1", "M")),
           (("0.1", "F"), ("0.1", "M")),
           (("0.1", "All"), ("straight", "All"))]

for numAxs in range(len(features_array)):
    fig = plt.figure(facecolor='w') 
    ax = sns.boxplot(data=individualResults_df, x="condition", y=features_array[numAxs], hue="Sex", hue_order=('All', 'F', 'M'), order=conditions)

    #annotator = Annotator(ax, pairs, data=individualResults_df, x="condition", y=features_array[numAxs], hue="Sex", hue_order=('All', 'F', 'M'))
    #annotator.configure(test='Mann-Whitney', text_format='star', loc='inside')
    #annotator.apply_and_annotate()
    
    fig.savefig(outputFolder + '/wingFeatures'+features_array[numAxs]+'.png', dpi=300, bbox_inches='tight')
    
    # Display figure with only the individual points
    fig = plt.figure(facecolor='w')
    # ax = sns.swarmplot(data=individualResults_df, x="condition", y=features_array[numAxs], hue="Sex", hue_order=('All', 'F', 'M'), order=conditions)
    ax = sns.stripplot(data=individualResults_df, x="condition", y=features_array[numAxs], hue="Sex", hue_order=('All', 'F', 'M'), order=conditions, dodge=True)
    fig.savefig(outputFolder + '/wingFeatures_swarm_'+features_array[numAxs]+'.png', dpi=300, bbox_inches='tight')
    

## FAi 6 analysis

In [ ]:
#https://www.w3schools.com/colors/colors_xkcd.asp

toDisplay_FAi_condition = []
toDisplay_FAi_sex = []
toDisplay_FAi_areas = []
toDisplay_FAi_width = []
toDisplay_sex = []
toDisplay_condition = []
toDisplay_areaNormalised = [];
for row in results_df.itertuples():
    toDisplay_FAi_condition.append(row.condition)
    if row.Sex == '':
        rowSex = 'All'
    else:
        rowSex = row.Sex
    toDisplay_FAi_sex.append(rowSex)
    toDisplay_FAi_areas.append(row.FAindex_Area)
    toDisplay_FAi_width.append(row.FAindex_Width)
    for value in row.areaNormalised_LR:
        toDisplay_sex.append(rowSex)
        toDisplay_condition.append(row.condition)
        toDisplay_areaNormalised.append(value)

df_toDisplay = pd.DataFrame(dict(Condition=toDisplay_FAi_condition, Sex = toDisplay_FAi_sex, FAi_Area=toDisplay_FAi_areas))
fig = plt.figure(facecolor='w')
ax = sns.barplot(data=df_toDisplay, x="Condition", y="FAi_Area", hue="Sex", hue_order=('All', 'F', 'M'), order=conditions)
fig.savefig(outputFolder + '/FAiIndex_Area.png', dpi=300, bbox_inches='tight')

df_toDisplay = pd.DataFrame(dict(Condition=toDisplay_FAi_condition, Sex = toDisplay_FAi_sex, FAi_Width=toDisplay_FAi_width))
fig = plt.figure(facecolor='w')
ax = sns.barplot(data=df_toDisplay, x="Condition", y="FAi_Width", hue="Sex", hue_order=('All', 'F', 'M'), order=conditions)
fig.savefig(outputFolder + '/FAiIndex_Width.png', dpi=300, bbox_inches='tight')

In [ ]:
df_toDisplay = pd.DataFrame(dict(Condition=toDisplay_condition, Sex = toDisplay_sex, AreaNormalised=toDisplay_areaNormalised))

fig = plt.figure(facecolor='w') 
ax = sns.boxplot(data=df_toDisplay, x="Condition", y="AreaNormalised", hue="Sex", hue_order=('All', 'F', 'M'), order=conditions)

#annotator = Annotator(ax, pairs, data=df_toDisplay, x="Condition", y="AreaNormalised", hue="Sex", hue_order=('All', 'F', 'M'))
#annotator.configure(test='Levene', text_format='star', loc='inside')
#annotator.apply_and_annotate()

fig.savefig(outputFolder + '/WingAreaNormalised.png', dpi=300, bbox_inches='tight')
#plt.ylim([-0.08, 0.08])

In [ ]:
for row in results_df.itertuples():
    print(row.condition + ', ' + row.Sex + ', n=' + str(row.n))